# Zero Shot Classification on Comics dataset

In [1]:
import os
import json
import torch
import pickle

import pandas as pd

from tqdm.notebook import tqdm
from pathlib import Path
from sklearn.metrics import classification_report
from transformers import AutoModelForCausalLM, AutoTokenizer

In [28]:
MODEL_NAME = "microsoft/Phi-3-mini-4k-instruct"

In [29]:
tokenizer = AutoTokenizer.from_pretrained(MODEL_NAME)
model = AutoModelForCausalLM.from_pretrained(MODEL_NAME)

Loading checkpoint shards:   0%|          | 0/2 [00:00<?, ?it/s]

In [30]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

Phi3ForCausalLM(
  (model): Phi3Model(
    (embed_tokens): Embedding(32064, 3072, padding_idx=32000)
    (embed_dropout): Dropout(p=0.0, inplace=False)
    (layers): ModuleList(
      (0-31): 32 x Phi3DecoderLayer(
        (self_attn): Phi3Attention(
          (o_proj): Linear(in_features=3072, out_features=3072, bias=False)
          (qkv_proj): Linear(in_features=3072, out_features=9216, bias=False)
          (rotary_emb): Phi3RotaryEmbedding()
        )
        (mlp): Phi3MLP(
          (gate_up_proj): Linear(in_features=3072, out_features=16384, bias=False)
          (down_proj): Linear(in_features=8192, out_features=3072, bias=False)
          (activation_fn): SiLU()
        )
        (input_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
        (resid_attn_dropout): Dropout(p=0.0, inplace=False)
        (resid_mlp_dropout): Dropout(p=0.0, inplace=False)
        (post_attention_layernorm): Phi3RMSNorm((3072,), eps=1e-05)
      )
    )
    (norm): Phi3RMSNorm((3072,), eps=1e-05)
  )
 

In [31]:
# def generate_prompt(text, labels):
#     prompt = f"Classify the following text into one of these categories. Restrict your answer to only one token. Do not generate any tokens other than the label: {', '.join(labels)}.\n\nText: {text}\n\nCategory:"
#     return prompt


In [73]:
def generate_prompt(text):
    prompt = f"""You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(es): anger, surprise, fear, disgust, sadness, joy, neutral.

Utterance: {text}

You must absolutely not generate any text or explanation other than the following JSON format: 
{{"utterance_emotion": "<predicted emotion classes for the utterance (str)>"}}. 

Answer:
"""

    #prompt = "### Task description: You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(s): anger, surprise, fear, disgust, sadness, joy, neutral. You must absolutely not generate any text or explanation other than the following JSON format: {utterance_emotion: <predicted emotion classes for the utterance (str)>}" + f"\n\n# Utterance:\n{text}\n\n# Result:\n"
    #prompt = f"You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(s): anger, surprise, fear, disgust, sadness, joy, neutral.\n\nUtterance: {text}\n\nYou must absolutely not generate any text or explanation other than the following JSON format: {utterance_emotion: <predicted emotion classes for the utterance (str)>}. Answer:"
    return prompt


In [74]:
# def generate_prompt(text, labels):
#     prompt = f"""You are an expert in emotion analysis. You are given a text from a comics book.\n\n### Here is the text: {text}\n\nYou must classify the emotion of this text into one or more of the following emotion labels: {labels}. Do not generate any explanation after the answer. ###Answer:###"""
#     return prompt

In [75]:
# def generate_prompt(text, labels):
#     prompt = (
#         f"You are an expert in emotion analysis. Below is a passage from a comic book.\n\n"
#         f"### Text:\n{text}\n\n"
#         f"Classify the emotion(s) of the text using one of the following labels: anger, surprise, fear, disgust, sadness, joy, neutral. Provide only the labels.\n\n"
#         f"### Answer:"
#     )
#     return prompt


In [76]:
text_to_classify = "The stock market saw a sharp rise today."
labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [77]:
generate_prompt(text_to_classify)

'You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(es): anger, surprise, fear, disgust, sadness, joy, neutral.\n\nUtterance: The stock market saw a sharp rise today.\n\nYou must absolutely not generate any text or explanation other than the following JSON format: \n{"utterance_emotion": "<predicted emotion classes for the utterance (str)>"}. \n\nAnswer:\n'

In [85]:
def classify_text(text, labels):
    # Create the prompt
    prompt = generate_prompt(text)
    
    # Tokenize and encode the prompt
    inputs = tokenizer(prompt, return_tensors="pt").to(model.device)

    # Generate output
    outputs = model.generate(**inputs, max_new_tokens=16, eos_token_id=tokenizer.eos_token_id, pad_token_id=tokenizer.eos_token_id)
    
    # Decode the generated output
    classification = tokenizer.decode(outputs[0], skip_special_tokens=True)
    
    # Extract the predicted category (after the prompt)
    #predicted_category = classification.split("Answer:")[-1].strip()
    return classification
    #return predicted_category

In [86]:
# text_to_classify = "The stock market saw a sharp rise today."
# labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [87]:
#predicted_category = classify_text(text_to_classify, labels)

In [88]:
#print(f"Predicted category: {predicted_category}")

### Read the data sets

In [89]:
df = pd.read_csv("/Utilisateurs/umushtaq/emotion_analysis_comics/zeroshot/datasets/comics_data_processed.csv")

In [90]:
df = df.drop(columns=[df.columns[0], df.columns[1]])

In [91]:
emotion_map = {
    'AN': 'anger',
    'DI': 'disgust',
    'FE': 'fear',
    'SA': 'sadness',
    'SU': 'surprise',
    'JO': 'joy'
}

In [92]:
def extract_emotions(row):

    emotion_str = row.emotion

    if emotion_str == 'Neutral':
        return ['neutral']

    emotions = emotion_str.split('-')
    tags = []

    for emotion in emotions:
        abbrev = emotion[:2]  # Get the abbreviation
        value_part = emotion[2:]  # Get the value part
        
        if abbrev in emotion_map and value_part.isdigit():
            value = int(value_part)
            if value > 0:
                tags.append(emotion_map[abbrev].lower())
        else:
            print(f"Warning: Skipping invalid emotion entry: '{emotion}'")
    return tags  

In [93]:
df['emotions_list'] = df.apply(lambda row: extract_emotions(row), axis=1)

In [94]:
#df = df[df['emotions_list'].apply(lambda x: len(x) == 1)].reset_index()

In [95]:
texts = df.utterance.tolist()

In [96]:
labels = ["anger", "surprise", "fear", "disgust", "sadness", "joy", "neutral"]

In [97]:
predictions = []

for i in tqdm(range(len(texts))):
    predicted_category = classify_text(texts[i], labels)
    predictions.append(predicted_category)

  0%|          | 0/5282 [00:00<?, ?it/s]

In [98]:
predictions

['You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(es): anger, surprise, fear, disgust, sadness, joy, neutral.\n\nUtterance: DID YOU HAVE TO ELECTROCUTE HER SO HARD?\n\nYou must absolutely not generate any text or explanation other than the following JSON format: \n{"utterance_emotion": "<predicted emotion classes for the utterance (str)>"}. \n\nAnswer:\n{"utterance_emotion": "anger"}\n\n\nInput:\n',
 'You are an expert sentiment analysis assistant that takes an utterance from a comic book and must classify the utterance into appropriate emotion class(es): anger, surprise, fear, disgust, sadness, joy, neutral.\n\nUtterance: IT\'S NOT LIKE I HAVE DIFFERENT SETTINGS.\n\nYou must absolutely not generate any text or explanation other than the following JSON format: \n{"utterance_emotion": "<predicted emotion classes for the utterance (str)>"}. \n\nAnswer:\n{"utterance_emotion": "anger"}\n

In [105]:
len(predictions)

5282

In [101]:
import re

In [102]:
preds = []

for prediction in predictions:

    match = re.search(r'Answer:\s*(\{.*?\})', prediction)

    if match:

        json_str = match.group(1)  # Extract the JSON object as a string
        try:
            # Parse the extracted JSON string
            parsed_json = json.loads(json_str)
            preds.append(parsed_json)
            #print("Extracted JSON:", parsed_json)
        except json.JSONDecodeError as e:
            print(f"Error parsing JSON: {e}")

In [103]:
preds

[{'utterance_emotion': 'anger'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'neutral'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'neutral'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'joy'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'neutral'},
 {'utterance_emotion': 'sadness'},
 {'utterance_emotion': 'neutral'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'anger'},
 {'utterance_emotion': 'neutral'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'neutral'},
 {'utterance_emotion': 'joy'},
 {'utterance_emotion': 'neutral'},
 {'utterance_emotion': 'surprise'},
 {'utterance_emotion': 'anger'},
 {'u

In [104]:
len(preds)

5279

In [99]:
preds = []

for prediction in predictions:
    try:
        preds.append(prediction.split("\n\n\n")[-1].split("Answer:")[1][1:])
    except:
        preds.append("neutral")

In [100]:
preds

['neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'neutral',
 'ne

In [67]:
len(preds)

5282

In [55]:
grounds = [x for [x] in df.emotions_list.tolist()]

In [56]:
for idx, (ground, pred) in enumerate(zip(grounds, preds)):

    if pred not in labels:
        del grounds[idx]
        del preds[idx]

In [57]:
print(classification_report(grounds, preds))

              precision    recall  f1-score   support

           '       0.00      0.00      0.00         0
    Answer:
       0.00      0.00      0.00         0
         The       0.00      0.00      0.00         0
       anger       0.63      0.27      0.38       601
         dis       0.00      0.00      0.00         0
     disgust       0.00      0.00      0.00        14
        fear       0.25      0.18      0.21       245
         joy       0.43      0.71      0.53       612
     neutral       0.22      0.33      0.26       271
         sad       0.00      0.00      0.00         0
     sadness       0.00      0.00      0.00       314
    surprise       0.39      0.36      0.38       338

    accuracy                           0.36      2395
   macro avg       0.16      0.15      0.15      2395
weighted avg       0.37      0.36      0.34      2395



/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Recall is ill-defined and being set to 0.0 in labels with no true samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
/Utilisateurs/umushtaq/.conda/envs/er_nb_env/lib/python3.11/site-packages/sklearn/metrics/_classification.py:1531: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifie